In [ ]:
import requests
from bs4 import BeautifulSoup
import traceback as tb
import json
import re

# 爬蟲程式進入點

In [ ]:
def ifoodie_crawler(limit_number):
    for offset in range(0,11800,limit_number):
        search_url = "https://ifoodie.tw/api/blog/?offset={}&limit={}&order_by=-date".format(offset,limit_number)
        print("[INFO]start crawling {}".format(search_url))
        get_all_restaurant_commend_info(search_url, offset, limit_number)

# 取得餐廳評論的程式

In [ ]:
def get_all_restaurant_commend_info(search_url, offset, limit_number):
    res = requests.get(search_url)
    soup = BeautifulSoup(res.text, 'lxml')
    pre_dict_list = soup.select('p')[0].text.split('response\": [')
    dict_list = pre_dict_list[1].split('''], "success"''')[0]

    pattern_split_dict = r'}, {'
    pre_dict_list = re.split(pattern_split_dict, dict_list)

    after_dict = '}'
    pre_dict = '{'

    json_objects_list = []
    pre_dict_list_size_int = len(pre_dict_list)
    
    first_restaurant_commend = '{}{}'.format(pre_dict_list[0],after_dict)
    try:
        json_object = json.loads(first_restaurant_commend)
        json_objects_list.append(json_object)
    except:
        print("[ERROR]first_restaurant_commend can't decode to JsonType")
        print("[ERROR-INFO]{}, object{}".format(search_url,1))

    for dict_number in range(1,pre_dict_list_size_int-1):
        restaurant_commend = '{}{}{}'.format(pre_dict,pre_dict_list[dict_number],after_dict)

        try:
            json_object = json.loads(restaurant_commend)
            json_objects_list.append(json_object)
        except:
            print("[ERROR]restaurant_commend can't decode to JsonType")
            print("[ERROR-INFO]{}, object{}".format(search_url,dict_number+1))
            continue

    last_restaurant_commend = '{}{}'.format(pre_dict,pre_dict_list[pre_dict_list_size_int-1])
    try:
        json_object = json.loads(last_restaurant_commend)
        json_objects_list.append(json_object)
    except:
        print("[ERROR]last_restaurant_commend can't decode to JsonType")
        print("[ERROR-INFO]{}, object{}".format(search_url,pre_dict_list_size_int))
    
    with open('ifoodie_json_list_{}to{}.json'.format(offset,offset+limit_number), 'a',encoding='utf8') as fa:
        data = json.dumps(json_objects_list, ensure_ascii=False)
        fa.write(data)
        print("[INFO]crawling {} was finish".format(search_url))

# 最後測試是否寫進去的東西讀出來之後可以用json格式來取值

In [ ]:
with open('ifoodie_json_list_1400to1600.json', 'r',encoding='utf8') as fr:
    data = json.loads(fr.read())

In [ ]:
data[7]

In [ ]:
data[7]["description"]

# 可以用字典檔的KV來取值了，OK!